<a href="https://colab.research.google.com/github/Rahmamouradsayed/Computer-Vision-/blob/main/Project_Computer_Vision_Rahma_Mourad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio opencv-python numpy
!apt install ffmpeg -y

In [ ]:
from google.colab import files
uploaded = files.upload()  #input_video.mp4

In [ ]:
import cv2
import torch
import numpy as np
from collections import deque

# Load YOLOv5s model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.4
model.classes = [0]  # Detect only persons

# Initialize tracking paths
paths = {}
max_len = 50
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]  # Red, Green, Blue

# Load input video
cap = cv2.VideoCapture("/content/input.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 25  # Fallback default FPS

# Define video writer for output
out = cv2.VideoWriter('output_tracked.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      (frame_width, frame_height))

person_ids = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    current_centroids = []

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        # Register the first 3 person IDs
        if len(person_ids) < 3:
            person_ids.append(len(person_ids))
            paths[person_ids[-1]] = deque(maxlen=max_len)

        # Track only the first 3 people
        pid = len(current_centroids)
        if pid >= 3:
            continue

        current_centroids.append((cx, cy))
        paths[pid].append((cx, cy))

        # Draw bounding box and label
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), colors[pid], 2)
        cv2.putText(frame, f'Person {pid}', (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[pid], 2)

    # Draw movement trails
    for pid, pts in paths.items():
        for i in range(1, len(pts)):
            cv2.line(frame, pts[i - 1], pts[i], colors[pid], 3)

    out.write(frame)

cap.release()
out.release()


In [ ]:
!ffmpeg -i output_tracked.mp4 -vcodec libx264 -crf 23 output_compressed.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output_compressed.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=720 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")
